![NVIDIA Logo](images/nvidia.png)

# Assessment: Identify Sources of Customer Complaints

In [1]:
from videos.walkthroughs import walkthrough_61 as walkthrough

In [2]:
walkthrough()

In this notebook you will complete a final workshop project and earn a certificate of competency for the workshop.

---

## Imports

We believe the following imports will be helpful in your work, but feel free to modify them as you deem necessary.

In [3]:
import json

from typing import List
from pprint import pprint

from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.pydantic_v1 import BaseModel, Field

from assessment_helper import run_assessment

---

## Create a Model Instance

For the assessment, you will be working with the same model you have been utilizing for the entirety of the workshop.

In [4]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## Assessment Objective

For the assessment, you will be provided with a small collection of 10 fictitious synthetically generated emails from customers of a mega retail store called BuyBuy. Each of these emails involves a customer from a specified store location either praising or complaining about a specific product they recently bought.

**Your objective is to create a LangChain chain that when invoked with the emails, will respond concisely with what category of product is most associated with negative customer sentiment, and also, which store location has the most customer complaints.**

---

## Customer Emails

Here we load the synthetic emails into a list called `emails`.

In [5]:
with open('data/emails.json', 'r') as f:
    emails = json.load(f)

As a sample, here is the first 3 emails in the collection.

In [6]:
for email in emails[:3]:
    print(email+'\n')

Dear BuyBuy, I just wanted to express how thrilled I am with the blender I purchased from your store in New York last week. It has made such a difference in my daily routine. The smoothies it makes are so smooth and creamy, and the powerful motor handles even the toughest ingredients like frozen fruit without any issues. I also really appreciate how easy it is to clean. This is by far one of the best kitchen gadgets I’ve ever owned. Thank you for offering such an excellent product! Best regards, Sarah

To whom it may concern, I am writing to share my frustration regarding the dining table I recently bought from BuyBuy in New York. When the table arrived, I was extremely disappointed to find scratches and dents all over the surface. On top of that, the assembly instructions were vague and difficult to follow, making the process even more stressful. For the price I paid, I expected a much higher level of quality. This experience has left me questioning whether I should shop at BuyBuy aga

---

## Product Categories

As stated above, we are interested in your chain being able to identify the **category of product** most associated with a negative sentiment. For example, if there were a complaint about a shirt, another about a jacket, and a third about some jeans, it would be fair to say that there were 3 complaints about **clothing**. If there were a complaint about a desk, and another about a couch, it would be fair to say that there were 2 complaints about **furniture**.

Asking an LLM to make such an identification is sensible since we are leveraging its language capabilities to help us gain insight where it might not otherwise be obvious.

On a more practical note, this means that you won't simply be able to count the number of occurences of a given product, but rather, need to ask the LLM to identify the correct `"category of product"`.

---

## Checking Your Work

Eventually, you will have created a LangChain chain that can be invoked with `emails` and then outputs the product category and store location most associated with customer complaints.

When you're ready, pass your chain into the provided `run_assessment` function, which will evaluate the behavior of your chain.

Here we create a mock chain just to show you how it can be invoked with `emails`, and, how it can be passed to `run_assessment`.

In [7]:
mock_prompt = '''\
Always and only respond with "The product category with the most negative sentiment is clothing.

The store location with the most negative sentiment is Dallas.

Ignore the following {emails}'''

In [8]:
mock_chain = ChatPromptTemplate.from_template(mock_prompt) | llm | StrOutputParser()

In [9]:
mock_chain.invoke(emails)

'The product category with the most negative sentiment is clothing.\n\nThe store location with the most negative sentiment is Dallas.'

In [10]:
run_assessment(mock_chain)

Passing emails into your chain...
Your chain completed successfully.

Checking whether your chain's summary correctly identified the product class with the most negative sentiments...
Your chain's summary did NOT correctly identify the product class with the most negative sentiment.

Checking whether your chain's summary correctly identified the location with the most negative sentiments...
Your chain's summary did NOT correctly identify the city with the most negative sentiment.

You did not successfully complete the assessment, please continue your work and try again.


---

## Your Work Here

There are any number of ways that you might approach this problem. We recommend you take some time to plan out how you plan to tackle it.

Remember, once you've completed a chain to your satisfaction, be sure to pass it into `run_assessment` to check your work. Once you've successfully completed the task, see the instructions below for how to generate your certificate of competency in the workshop.

In [15]:
import json
import re
import logging
from collections import Counter, defaultdict
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableLambda

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

with open('data/emails.json', 'r') as f:
    emails = json.load(f)

email_prompt = ChatPromptTemplate.from_template(
    "Return ONLY a JSON object with keys 'category', 'location', and 'sentiment' "
    "(Positive or Negative) based on the following customer email.\n\nEmail:\n{email}"
)

parser = StrOutputParser()
analysis_chain = RunnableLambda(lambda e: {"email": e}) | email_prompt | llm | parser

def extract_json(raw_text):
    match = re.search(r'\{.*\}', raw_text, re.DOTALL)
    if match:
        return match.group(0)
    else:
        raise ValueError("No JSON object found in response.")

category_sentiment = defaultdict(Counter)
location_sentiment = defaultdict(Counter)
pair_sentiment = defaultdict(lambda: Counter())
all_categories = Counter()
success_count = 0
fail_count = 0

for email in emails:
    result_raw = analysis_chain.invoke(email)
    try:
        clean_json = extract_json(result_raw)
        result = json.loads(clean_json)
        category = result['category'].strip().lower()
        location = result['location'].strip().lower()
        sentiment = result['sentiment'].strip().lower()
        category_sentiment[category][sentiment] += 1
        location_sentiment[location][sentiment] += 1
        pair_sentiment[(category, location)][sentiment] += 1
        all_categories[category] += 1
        success_count += 1
    except Exception as e:
        logger.warning(f"Failed to parse result: {e}\nRaw: {result_raw}")
        fail_count += 1

worst_category = max(category_sentiment.items(), key=lambda x: x[1]['negative'])[0].title() if category_sentiment else "No data"
worst_location = max(location_sentiment.items(), key=lambda x: x[1]['negative'])[0].title() if location_sentiment else "No data"
top_complaint_pair = max(pair_sentiment.items(), key=lambda x: x[1]['negative'])[0] if pair_sentiment else ("No data", "No data")

print("\n===== FINAL REPORT =====")
print(f"Product category with most negative sentiment: {worst_category}")
print(f"Store location with most complaints: {worst_location}")
print(f"Top complaint source (category + store): {top_complaint_pair[0].title()} at {top_complaint_pair[1].title()}")
print(f"Emails processed successfully: {success_count}")
print(f"Emails failed to process: {fail_count}")
print("\n--- Category Breakdown ---")
for cat, counts in category_sentiment.items():
    total = counts['positive'] + counts['negative']
    print(f"{cat.title()}: {total} total ({counts['positive']} positive, {counts['negative']} negative)")
print("\n--- Store Breakdown ---")
for loc, counts in location_sentiment.items():
    total = counts['positive'] + counts['negative']
    print(f"{loc.title()}: {total} total ({counts['positive']} positive, {counts['negative']} negative)")
print("=========================")




===== FINAL REPORT =====
Product category with most negative sentiment: Furniture
Store location with most complaints: New York
Top complaint source (category + store): Furniture at New York
Emails processed successfully: 10
Emails failed to process: 0

--- Category Breakdown ---
Kitchen: 1 total (1 positive, 0 negative)
Furniture: 3 total (0 positive, 3 negative)
Shoes: 1 total (1 positive, 0 negative)
Coffee Maker: 1 total (1 positive, 0 negative)
Customer Service: 1 total (0 positive, 1 negative)
Appliances: 1 total (1 positive, 0 negative)
Product Defect: 1 total (0 positive, 1 negative)
Electronics: 1 total (1 positive, 0 negative)

--- Store Breakdown ---
New York: 5 total (1 positive, 4 negative)
Dallas: 1 total (1 positive, 0 negative)
Oakland: 2 total (1 positive, 1 negative)
Downtown Dallas: 1 total (1 positive, 0 negative)
Buybuy: 1 total (1 positive, 0 negative)


---

## Get Certificate for the Workshop

Assuming you've received a message from `run_assessment` that you successfully completed the assessment, your ready to generate a certificate of competency for the workshop.

In your web browser, return to the page where you launched this interactive environment and click the check-mark `ASSESS TASK` button (see the screenshot below). After a few seconds you will get a congratulatory message, after which you can visit your [personal DLI learning page](https://learn.nvidia.com/my-learning) and view your certificate.

![assess](images/assess.png)